# Conociendo los RDDs


En este notebook trabajaremos con los RDDs, que forman parte del Spark Core.La implementación de Spark Core es un **RDD (Resilient Distributed Dataset)** que es una colección de datos distribuidos en diferentes nodos del clúster que se procesan en paralelo.

Utilizaremos la API de PySpark, pero los conceptos aplican por igual a todas las APIs (Scala, R, etc)

### Inicialización de Spark en Notebooks

In [2]:
# Install spark-related dependencies
!wget -q  https://apache.osuosl.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
!tar xf spark-3.5.4-bin-hadoop3.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

In [4]:
import findspark
findspark.init() #para inicializar

import pandas as pd
import pyspark

### Crear el SparkSession y el SparkContext

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_training')\
        .getOrCreate() #devuelve una sesion existente sino existe la crea

sc = spark.sparkContext

**local[*]**

* "local" significa que Spark se ejecutará en modo local, en tu máquina actual, en lugar de en un clúster de máquinas. Esto es útil para desarrollo y pruebas.

* El [*] le dice a Spark que use todos los núcleos de CPU disponibles en tu máquina. El asterisco actúa como un comodín, determinando dinámicamente el número de núcleos a utilizar.

In [ ]:
spark = SparkSession.builder.getOrCreate() #genera lo mismo que antes solo que con los valores por default
sc = spark.sparkContext

### Crear un RDD de una colección

In [7]:
num = [1,2,3,4,5]

num_rdd = sc.parallelize(num) #función para paralelizar
num_rdd.collect() #con collect, recogemos toda la lista de números

[1, 2, 3, 4, 5]

# Transformaciones
* Como sabemos, las Transformaciones son de naturaleza perezosa y no se ejecutarán hasta que se ejecute una Acción sobre ellas.
* Intentemos comprender las distintas transformaciones disponibles.

Para mas informacion, puedes apoyarte del siguiente link:
* https://keepcoding.io/blog/transformaciones-y-acciones-en-spark/


### map
* Esto mapeará su entrada a alguna salida basada en la función especificada en la función

In [8]:
double_rdd = num_rdd.map(lambda x : x * 2)
double_rdd.collect()

[2, 4, 6, 8, 10]

**collect()** toma todos los datos distribuidos en el clúster Spark (o en tu máquina local si estás trabajando en modo local) y los reúne en una sola lista en tu programa, donde puedes acceder a ellos y trabajar con ellos directamente.

**Importante:**

* collect() es una acción, lo que significa que desencadena la ejecución de todas las transformaciones previas que se hayan aplicado al RDD.
* Se debe usar con precaución en RDDs grandes, ya que traer todos los datos al programa principal puede consumir mucha memoria y tiempo. Es preferible usar otras acciones como **take()** para obtener una muestra de los datos si solo necesitas una parte de ellos.

### filtro
* Para filtrar los datos en función de una determinada condición. Intentemos encontrar los números pares de num_rdd.

In [14]:
even_rdd = num_rdd.filter(lambda x : x % 2 == 0) #filtro por los elementos pares del rdd
even_rdd.collect()

[2, 4]

### distinct
* Esto devolverá elementos distintos de un RDD.

In [30]:
rdd1 = sc.parallelize([10, 11, 11, 13, 11, 10, 12])

# la función distinct sirve para eliminar elementos duplicados de un RDD.
dist_rdd = rdd1.distinct()

#dist_rdd = dist_rdd.sortBy(lambda x: x) #por si queremos los elementos ordenados
dist_rdd.collect()

[10, 12, 11, 13]

### reduceByKey
* **Agrupación por clave**: reduceByKey agrupa todos los elementos del RDD que tienen la misma clave. En este caso, agrupará los elementos con clave "a", los elementos con clave "b" y los elementos con clave "c".

* **Reducción de valores**: Después de la agrupación, reduceByKey aplica la función "lambda x, y : x + y" a los valores asociados a cada clave. Esta función simplemente suma los dos valores (x e y) que se le pasan.

In [26]:
pairs = [ ("a", 8), ("b", 3), ("c", 3), ("a", 5), ("b", 1), ("c", 4)]
pair_rdd = sc.parallelize(pairs)

output = pair_rdd.reduceByKey(lambda x, y : x + y)

result = output.sortByKey().collect()
print(*result, sep='\n')

('a', 13)
('b', 4)
('c', 7)


### sortByKey
* Esta función realizará la clasificación en un par (clave, valor) RDD basado en las claves. De forma predeterminada, la clasificación se realizará en orden ascendente.

In [32]:
pairs = [ ("a", 5), ("d", 7), ("c", 2), ("b", 3)]
raw_rdd = sc.parallelize(pairs)

sortkey_rdd = raw_rdd.sortByKey() #ascending=True
#sortkey_rdd = raw_rdd.sortByKey(ascending=False)
result = sortkey_rdd.collect()
print(*result,sep='\n')

('a', 5)
('b', 3)
('c', 2)
('d', 7)


# Acciones

* Las acciones son operaciones en RDD que se ejecutan inmediatamente. Mientras que las transformaciones devuelven otro RDD, las acciones devuelven estructuras de datos nativas

### count
* Esto contará el número de elementos en el RDD dado.

In [34]:
num = sc.parallelize([1,2,4,5,2])
num.count()

5

### first
* Esto devolverá el primer elemento del RDD dado.

In [35]:
num.first()

1

### Collect
* Esto devolverá todos los elementos para el RDD dado.


In [ ]:
num.collect()

[1, 2, 4, 5, 2]

### Take
* Esto devolverá el número de elementos especificados.

In [37]:
num.take(3)

[1, 2, 4]

In [38]:
sc.stop()